In [16]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold

In [2]:
data = pd.read_csv('AmesHousing.tsv',delimiter='\t')

Structure of code is like that I am defining 3 functions, at the end I will obtain rmse value. Eventually I will updating these 3 functions to lower down the rmse value. Lets see how much I can lower down rmse. 

In [3]:
def transform_features(df):
    return(df)

def select_features(df):
    return(df[['Gr Liv Area','SalePrice']])

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    cols = train.select_dtypes(['int32','int64','float32','float64']).drop(['SalePrice'],axis=1).columns 
    lr = LinearRegression()
    lr.fit(train[cols],train['SalePrice'])
    predictions = lr.predict(test[cols])
    rmse = mean_squared_error(test['SalePrice'],predictions)**0.5
    return(rmse)

transformed = transform_features(data)
selected = select_features(transformed)
rmse = train_and_test(selected)
rmse

57088.25161263909

Firstly, I will update transform() feature function, according to these steps - 
1. Removing columns with more than 25% null values.
2. For the remaining columns, filling the null values with respective mode.
3. Making new columns 'years_before_sale' and 'years_since_remod'.
4. Removing negative rows.
5. Removing columns like 'Order' and 'PID', as they have no link with ML.
6. Removing columns like 'Mo Sold', 'Sale Type', etc that have relation with target column,'SalePrice'.

In [13]:
def transform_features(df):
    mv = df.isnull().sum()
    df.drop(mv[mv>0.25*2930].index,axis=1)
    replace_val = df.mode().to_dict(orient='records')[0]
    df = df.fillna(replace_val)
    df['years_before_sale'] = df['Yr Sold'] - df['Year Built']
    df['years_since_remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df = df.drop([1702,2180,2181],axis=0)
    df = df.drop(['Yr Sold','Mo Sold','Order','PID','Sale Type','Sale Condition'],axis=1)
    return(df)

def select_features(df):
    return(df[['Gr Liv Area','SalePrice']])

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    cols = train.select_dtypes(['int32','int64','float32','float64']).drop(['SalePrice'],axis=1).columns 
    lr = LinearRegression()
    lr.fit(train[cols],train['SalePrice'])
    predictions = lr.predict(test[cols])
    rmse = mean_squared_error(test['SalePrice'],predictions)**0.5
    return(rmse)

transformed = transform_features(data)
selected = select_features(transformed)
rmse = train_and_test(selected)
rmse

55275.367312413066

In the code below, we gonna update select_features(). Steps - 
1. Removing columns which has less corr than 0.4 with SalePrice column.
2. Removing text columns which has less than 10 unique values. 
3. Converting the remaining text columns to category datatype.
4. Finally removing those original text columns as well and only keeping their dummies. 

In [15]:
def transform_features(df):
    mv = df.isnull().sum()
    df.drop(mv[mv>0.25*2930].index,axis=1)
    replace_val = df.mode().to_dict(orient='records')[0]
    df = df.fillna(replace_val)
    df['years_before_sale'] = df['Yr Sold'] - df['Year Built']
    df['years_since_remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df = df.drop([1702,2180,2181],axis=0)
    df = df.drop(['Yr Sold','Mo Sold','Order','PID','Sale Type','Sale Condition'],axis=1)
    return(df)

def select_features(df):
    x = np.abs(df.corr()['SalePrice'])
    low_corr = x[x < 0.4].index
    df = df.drop(low_corr,axis=1)
    
    object_cols = df.select_dtypes('object').columns
    unique = list()
    non_unique = list()
    for col in object_cols:
        if len(df[col].value_counts().index) < 10:
            unique.append(col)
        else:
            non_unique.append(col)
    df = df.drop(non_unique,axis=1)
    
    for col in unique:
        df[col] =  df[col].astype('category')
    
    df = pd.concat([df , pd.get_dummies(df.select_dtypes(include=['category']))],axis=1)
    df = df.drop(unique,axis=1)
    
    return(df)

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    cols = train.select_dtypes(['int32','int64','float32','float64']).drop(['SalePrice'],axis=1).columns 
    lr = LinearRegression()
    lr.fit(train[cols],train['SalePrice'])
    predictions = lr.predict(test[cols])
    rmse = mean_squared_error(test['SalePrice'],predictions)**0.5
    return(rmse)

transformed = transform_features(data)
selected = select_features(transformed)
rmse = train_and_test(selected)
rmse

36565.859382318966

In [51]:
def transform_features(df):
    mv = df.isnull().sum()
    df.drop(mv[mv>0.25*2930].index,axis=1)
    replace_val = df.mode().to_dict(orient='records')[0]
    df = df.fillna(replace_val)
    df['years_before_sale'] = df['Yr Sold'] - df['Year Built']
    df['years_since_remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df = df.drop([1702,2180,2181],axis=0)
    df = df.drop(['Yr Sold','Mo Sold','Order','PID','Sale Type','Sale Condition'],axis=1)
    return(df)

def select_features(df):
    x = np.abs(df.corr()['SalePrice'])
    low_corr = x[x < 0.4].index
    df = df.drop(low_corr,axis=1)
    
    object_cols = df.select_dtypes('object').columns
    unique = list()
    non_unique = list()
    for col in object_cols:
        if len(df[col].value_counts().index) < 10:
            unique.append(col)
        else:
            non_unique.append(col)
    df = df.drop(non_unique,axis=1)
    
    for col in unique:
        df[col] =  df[col].astype('category')
    
    df = pd.concat([df , pd.get_dummies(df.select_dtypes(include=['category']))],axis=1)
    df = df.drop(unique,axis=1)
    
    return(df)

def train_and_test(df,k=0):
    cols = df.select_dtypes(['int32','int64','float32','float64']).drop(['SalePrice'],axis=1).columns
    if k==0:
        train = df[0:1460]
        test = df[1460:]
        lr = LinearRegression()
        lr.fit(train[cols],train['SalePrice'])
        predictions = lr.predict(test[cols])
        rmse = mean_squared_error(test['SalePrice'],predictions)**0.5
        return(rmse)
    
    elif k==1:
        df = df.sample(frac=1)
        t1 = df[0:1460]
        t2 = df[1460:]
        lr = LinearRegression()
        
        lr.fit(t1[cols],t1['SalePrice'])
        predictions1 = lr.predict(t2[cols])
        rmse1 = mean_squared_error(t2['SalePrice'],predictions1)**0.5
        
        lr.fit(t2[cols],t2['SalePrice'])
        predictions2 = lr.predict(t1[cols])
        rmse2 = mean_squared_error(t1['SalePrice'],predictions2)**0.5
        print(rmse1, rmse2)
        return((rmse1+rmse2)/2)
    
    else:
        kf = KFold(n_splits = k, shuffle=True)
        lr = LinearRegression()
        mses = cross_val_score(lr,df[cols],df['SalePrice'],scoring='neg_mean_squared_error',cv=kf)
        avg_rmse = np.mean(np.sqrt(np.abs(mses)))
        return(avg_rmse)

transformed = transform_features(data)
selected = select_features(transformed)
rmse = train_and_test(selected,k=4)
rmse

33294.243133937285

In [50]:
selected

,Overall Qual,Year Built,Year Remod/Add,Mas Vnr Area,Total Bsmt SF,1st Flr SF,Gr Liv Area,Full Bath,Garage Cars,Garage Area,...,Pool QC_Gd,Pool QC_TA,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Misc Feature_Gar2,Misc Feature_Othr,Misc Feature_Shed,Misc Feature_TenC
0,6,1960,1960,112.0,1080.0,1656,1656,1,2.0,528.0,...,0,0,0,0,1,0,0,0,1,0
1,5,1961,1961,0.0,882.0,896,896,1,1.0,730.0,...,0,0,0,0,1,0,0,0,1,0
2,6,1958,1958,108.0,1329.0,1329,1329,1,1.0,312.0,...,0,0,0,0,1,0,1,0,0,0
3,7,1968,1968,0.0,2110.0,2110,2110,2,2.0,522.0,...,0,0,0,0,1,0,0,0,1,0
4,5,1997,1998,0.0,928.0,928,1629,2,2.0,482.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,6,1984,1984,0.0,1003.0,1003,1003,1,2.0,588.0,...,0,0,1,0,0,0,0,0,1,0
2926,5,1983,1983,0.0,864.0,902,902,1,2.0,484.0,...,0,0,0,0,1,0,0,0,1,0
2927,5,1992,1992,0.0,912.0,970,970,1,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
2928,5,1974,1975,0.0,1389.0,1389,1389,1,2.0,418.0,...,0,0,0,0,1,0,0,0,1,0


<generator object _BaseKFold.split at 0x00000166F8A623C8>